In [1]:
import math
import numpy as np
import scipy as sp
import time as ti

from matplotlib import pyplot as plt

%matplotlib inline

# Finite Difference

In [2]:
def diff_coeff(l, m, n):
    
    """
    
    This function generates the 'n-th' coefficient of a 1-D finite difference
    scheme for an arbitrary order 'l' with an offset 'm'. The 'offset' is 
    defined as the point within the stencil where the approximation for the
    derivative is calculated. For instance, a finite difference scheme of an 
    l-point stencil with m=(l-1)/2 and l is odd, is called a 'central' finite 
    difference. For the 'forward' and 'backward' difference schemes, m is 0 
    and l-1, respectively.
    
    """
    
    def factorial(k):
        if type(k) is not int:
            print("Invalid input! The function 'factorial' only accepts non-negative integers.")
        elif k < 0:
            return math.inf
        elif k == 0:
            return 1
        else:
            return k*factorial(k-1)
    
    coeff = 0
    if (m >= l) or (n >= l):
        print("Invalid inputs! The function 'diff_coeff' requires 0 < m < l and 0 < n < l.")
        return None
    elif m != n:
        coeff = (-1)**(m+n)/(m-n) * factorial(m)*factorial(l-m-1)/factorial(n)/factorial(l-n-1)
    else:
        for i in range(l):
            if i == m:
                continue
            else:
                coeff += -diff_coeff(l, m, i)
    
    return coeff

In [3]:
def bounded_diff(M, m=1):
    
    """
    
    This function generates the difference matrix for a bounded
    domain of M grid points. with a difference scheme of order 2m+1.
    Suggestion: use xrange with either bound.
    
    """
    
    matrix = np.zeros((M, M))
    l = 2*m+1
    
    if (M < l) or (type(M) is not int) or (type(m) is not int):
        print("Invalid inputs! Expecting integers for M and m, where M >= 2m+1.")
        return None
    else:
        for i in range(M):
            for j in range(M):
                if (i < m) and (j < l):
                    matrix[i, j] = diff_coeff(l, i, j)
                elif (M-i <= m) and (M-j <= l):
                    matrix[i, j] = diff_coeff(l, i+l-M, j+l-M)
                elif np.abs(j-i) <= m:
                    matrix[i, j] = diff_coeff(l, m, j-i+m)
                else:
                    continue
    
    return matrix

In [4]:
def periodic_diff(M, m=1):
    
    """
    
    This function generates the difference matrix for a periodic
    domain of M grid points. with a difference scheme of order 2m+1.
    Suggestion: use xrange 'False' bound.
    
    """
    
    matrix = np.zeros((M, M))
    l = 2*m+1
    
    if (M < l) or (type(M) is not int) or (type(m) is not int):
        print("Invalid inputs! Expecting integers for M and m, where M >= 2m+1.")
        return None
    else:
        for i in range(M):
            for j in range(M):
                if np.abs(j-i) <= m:
                    matrix[i, j] = diff_coeff(l, m, j-i+m)
                elif M-np.abs(j-i) <= m:
                    matrix[i, j] = diff_coeff(l, m, int(np.sign(j-i)*(np.abs(j-i)-M)+m))
                else:
                    continue
    
    return matrix

In [5]:
def xrange(i, f, n, bound=True):
    
    """
    
    
    
    """
    if bound:
        step_size = (f-i)/(n-1)
        init = i
    else:
        step_size = (f-i)/(n+1)
        init = i + step_size
        
    result = np.zeros(n)
    for step in range(n):
        result[step] = init + step*step_size
    
    return result

In [6]:
def osc(x):
    return x**2/2

In [7]:
def make_ham(bound, potential, dim=1024, order=1):
    # Region to solve
    X = 2*bound
    dX = X/(dim+1)
    x = np.linspace(-bound+dX, bound-dX, dim)
    
    return x